In [ ]:
import sys
sys.path.append('../')
sys.path.append('../src/')

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

from src.utils import load_moons, load_monk1, load_mnist, load_cup
from src.network import Network
from src.activations import ReLU, Tanh, Sigmoid
from src.losses import MeanSquaredError
from src.metrics import BinaryAccuracy, MulticlassAccuracy, MeanEuclideanError
from src.validation import kfold_cv
from src.callbacks import EarlyStopping
from src.regularizers import L2
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, x_test, y_train, y_val, y_test = load_moons(validation=True, noise=0.2)

In [ ]:
model = Network(2)
model.add_layer(8, ReLU())
model.add_layer(1, Tanh())

# create stats
stats = model.train(train=(x_train, y_train), validation=(x_val, y_val), metric=BinaryAccuracy(), loss=MeanSquaredError(), epochs=500, eta=0.01, nesterov=0.5)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.lineplot(stats['train_loss'], ax=axs[0])
sns.lineplot(stats['val_loss'], ax=axs[0])
sns.lineplot(stats['train_acc'], ax=axs[1])
sns.lineplot(stats['val_acc'], ax=axs[1])

In [ ]:
predictions = model.multiple_outputs(x_test)

accuracy_score(y_test.flatten(), np.round(predictions.flatten()))

In [ ]:
x_train, x_val, x_test, y_train, y_val, y_test = load_monk1()

In [ ]:
model = Network(17)
model.add_layer(6, ReLU())
model.add_layer(1, Tanh())
stats = model.train((x_train, y_train), (x_test, y_test), metric=BinaryAccuracy(), loss=MeanSquaredError(), epochs=1000, nesterov=0.5)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.lineplot(stats['train_loss'], ax=axs[0])
sns.lineplot(stats['val_loss'], ax=axs[0])
sns.lineplot(stats['train_acc'], ax=axs[1])
sns.lineplot(stats['val_acc'], ax=axs[1])


In [ ]:
x_train, x_test, y_train, y_test = load_mnist()

In [ ]:
model = Network(64)
model.add_layer(16, ReLU())
model.add_layer(10, Sigmoid(0.001))
stats = model.train(x_train, y_train, x_test, y_test, metric=MulticlassAccuracy(), loss=MeanSquaredError(), epochs=500)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.lineplot(stats['train_loss'], ax=axs[0])
sns.lineplot(stats['val_loss'], ax=axs[0])
sns.lineplot(stats['train_acc'], ax=axs[1])
sns.lineplot(stats['val_acc'], ax=axs[1])

In [ ]:
def load_cup(test_size=0.2):
    df = pd.read_csv("../data/cup/cup.train", comment="#", index_col='id', skipinitialspace=True)
    scaler = MinMaxScaler()
    scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    x_train, x_val, y_train, y_val = train_test_split(scaled.drop(["ty", 'tx'], axis=1).values, scaled[['tx','ty']].values, test_size=0.25, random_state=42)
    x_train = np.expand_dims(x_train, 2)
    x_val = np.expand_dims(x_val, 2)
    y_train = np.expand_dims(y_train, 2)
    y_val = np.expand_dims(y_val, 2)

    return x_train, x_val, y_train, y_val

In [23]:
x_train, x_val, y_train, y_val = load_cup()
x_train.shape

(1119, 9, 1)

In [25]:
model = Network(9)
model.add_layer(16, ReLU())
model.add_layer(8, ReLU())
model.add_layer(2, Sigmoid())

stats = model.train((x_train, y_train), (x_val, y_val), metric=MeanEuclideanError(), loss=MeanSquaredError(), epochs=1000, nesterov = 0.9,
                    callbacks=[])

NameError: name 'L2' is not defined

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.lineplot(stats['train_loss'], ax=axs[0])
sns.lineplot(stats['val_loss'], ax=axs[0])
sns.lineplot(stats['train_acc'], ax=axs[1])
sns.lineplot(stats['val_acc'], ax=axs[1])


### Test Load_Cup with kfold cross validation.

In [4]:
x, x_test, y, y_test = load_cup()
x.shape, y.shape

((1119, 9, 1), (1119, 2, 1))

In [7]:
model = Network(9,)
model.add_layer(8, ReLU())
model.add_layer(8, ReLU())
model.add_layer(2, Sigmoid())

kfold_cv(model, x, y, k=5, metric=MeanEuclideanError(), loss=MeanSquaredError(), epochs=1000, eta=0.01, verbose=True, callbacks=[EarlyStopping(patience=100)])

Training:  14%|█▎        | 136/1000[, loss=0.0145, val_loss=0.0145, val_acc=0.15] 

KeyboardInterrupt: 